<a href="https://colab.research.google.com/github/giangdip2410/Learn-NLP/blob/master/NLP_L1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
data_pd = pd.read_csv("IMDB Dataset.csv")

In [6]:
data_pd.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
from keras.preprocessing.text import text_to_word_sequence
#Using emotional dictionary
positive_words = ["good", "nice", "great", "interesting", "like", "wonderful","best","love"]
count_correct = 0
for review,sentiment in data_pd.values:
  words = text_to_word_sequence(review)
  result = "negative"
  for word in words:
    if word in positive_words:
      result = "positive"
  if sentiment == result:
    count_correct+=1
    

print("Acc:",count_correct/len(data_pd))

Acc: 0.5331


In [0]:
#Using bag of word vectors
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_pd["review"].values)

In [9]:
len(vectorizer.get_feature_names())

101895

In [10]:
vectorizer = CountVectorizer(max_features=200)
X = vectorizer.fit_transform(data_pd["review"].values)
X.toarray()

array([[0, 1, 0, ..., 3, 0, 1],
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 2, 0, ..., 1, 0, 0],
       [0, 1, 0, ..., 1, 0, 0]])

In [11]:
data_pd["label"] = data_pd["sentiment"].map({'positive': 1, 'negative': 0})
data_pd.head()

,review,sentiment,label
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1


In [0]:
y = data_pd["label"].values

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=42, stratify=y)

import numpy as np

print (np.sum(y)/len(y))
print (np.sum(y_train)/len(y_train))

print (X_train.shape,X_test.shape)

0.5
0.5
(15000, 200) (35000, 200)


In [14]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train,y_train) 

#Training dense 


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [0]:
y_test_predict = clf.predict(X_test)

In [16]:
print("Acc:",(1 - np.sum(np.abs(y_test_predict- y_test))/len(y_test_predict)))

Acc: 0.7626


In [0]:
#using n-gram
vectorizer = CountVectorizer(max_features=200,ngram_range=(1,2),stop_words=["the","that"])
X = vectorizer.fit_transform(data_pd["review"].values)
X = X.toarray()
#vectorizer.get_feature_names()

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=40, stratify=y)

In [20]:
from sklearn import svm
clf = svm.SVC(gamma='scale')
clf.fit(X_train,y_train) 

KeyboardInterrupt: ignored

In [29]:
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
from keras.layers import *
from keras.models import Model

input_layer = Input(shape=(200,))
dense_layer1 = Dense(64, activation="sigmoid")(input_layer)
dense_layer2 = Dense(24, activation="sigmoid")(dense_layer1)
dense_layer3 = Dense(2, activation="sigmoid")(dense_layer2)
model = Model(inputs =input_layer, outputs =dense_layer3)
model.compile(loss='binary_crossentropy', optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, validation_data =(X_test,y_test), epochs=10, batch_size=100)




Train on 15000 samples, validate on 35000 samples
Epoch 1/10





15000/15000 [==============================] - 2s 106us/step - loss: 0.6610 - acc: 0.6366 - val_loss: 0.5829 - val_acc: 0.7342
Epoch 2/10
15000/15000 [==============================] - 1s 38us/step - loss: 0.5267 - acc: 0.7564 - val_loss: 0.5006 - val_acc: 0.7616
Epoch 3/10
15000/15000 [==============================] - 1s 39us/step - loss: 0.4872 - acc: 0.7694 - val_loss: 0.4881 - val_acc: 0.7651
Epoch 4/10
15000/15000 [==============================] - 1s 37us/step - loss: 0.4763 - acc: 0.7735 - val_loss: 0.4868 - val_acc: 0.7643
Epoch 5/10
15000/15000 [==============================] - 1s 37us/step - loss: 0.4732 - acc: 0.7745 - val_loss: 0.4904 - val_acc: 0.7601
Epoch 6/10
15000/15000 [==============================] - 1s 37us/step - loss: 0.4699 - acc: 0.7771 - val_loss: 0.4838 - val_acc: 0.7658
Epoch 7/10
15000/15000 [==============================] - 1s 37us/step - loss: 0.4657 - acc: 0.7797 - val_loss: 0.4842 